In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root = './data',
    train = True,
    transform = transforms.ToTensor(),
    download = True
)
test_dataset = torchvision.datasets.MNIST(
    root = './data',
    train = False,
    transform = transforms.ToTensor(),
)

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
)
test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
)

In [8]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2), # 1*28*28 -> 16*28*28  output_size = (input_size + 2*padding - kernel_size)/stride + 1
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2) # 16*28*28 -> 16*14*14 output_size = (input_size - kernel_size)/stride + 1
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2), # 16*14*14 -> 32*14*14
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2) # 32*14*14 -> 32*7*7
        )
        self.fc = nn.Linear(7*7*32,num_classes)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0),-1) # flatten. the output of conv2 to (batch_size, 32*7*7)
        out = self.fc(out)
        return out


In [9]:
model = ConvNet(num_classes).to(device)
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [10]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch+1,num_epochs,i+1,total_step,loss.item()
            ))

Epoch [1/5], Step [100/600], Loss: 0.1246
Epoch [1/5], Step [200/600], Loss: 0.0777
Epoch [1/5], Step [300/600], Loss: 0.0165
Epoch [1/5], Step [400/600], Loss: 0.0307
Epoch [1/5], Step [500/600], Loss: 0.0391
Epoch [1/5], Step [600/600], Loss: 0.0501
Epoch [2/5], Step [100/600], Loss: 0.0609
Epoch [2/5], Step [200/600], Loss: 0.0203
Epoch [2/5], Step [300/600], Loss: 0.0037
Epoch [2/5], Step [400/600], Loss: 0.0693
Epoch [2/5], Step [500/600], Loss: 0.0692
Epoch [2/5], Step [600/600], Loss: 0.0344
Epoch [3/5], Step [100/600], Loss: 0.0175
Epoch [3/5], Step [200/600], Loss: 0.0514
Epoch [3/5], Step [300/600], Loss: 0.0107
Epoch [3/5], Step [400/600], Loss: 0.0044
Epoch [3/5], Step [500/600], Loss: 0.0017
Epoch [3/5], Step [600/600], Loss: 0.0677
Epoch [4/5], Step [100/600], Loss: 0.0027
Epoch [4/5], Step [200/600], Loss: 0.0900
Epoch [4/5], Step [300/600], Loss: 0.0267
Epoch [4/5], Step [400/600], Loss: 0.0132
Epoch [4/5], Step [500/600], Loss: 0.0222
Epoch [4/5], Step [600/600], Loss:

In [11]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the 10000 test images: {}%'.format(100*correct/total))

Accuracy of the model on the 10000 test images: 98.26%
